In [2]:
from langchain_community.llms import Ollama
from langchain.agents import tool
from langchain.agents import tool, initialize_agent, AgentType, Tool
import sys
import os


sys.path.append('../../bluesky')  # Adjust the path as necessary


# Now you can import bluesky modules

sys.path.append(os.path.abspath('../'))
import os
import streamlit as st

import bluesky

from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

from langchain.agents.format_scratchpad.openai_tools import (

    format_to_openai_tool_messages,

)


from langchain_community.tools.tavily_search import TavilySearchResults

from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI

from openai import OpenAI

from io import StringIO

from contextlib import contextmanager
import time

from dotenv import load_dotenv, find_dotenv

import chromadb.utils.embedding_functions as embedding_functions

import chromadb

from bluesky.network.client import Client

from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper

from prompts.prompts import conflict_prompt


from langchain.output_parsers import ResponseSchema, StructuredOutputParser

from langchain_experimental.utilities import PythonREPL

from langchain.agents import Tool

Using Python-based geo functions


In [3]:
# Load the .env file
load_dotenv(find_dotenv())

True

In [4]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-large"
)
vectordb_path = 'C:/Users/justa/OneDrive/Desktop/Developer/LLM-Enhanced-ATM/llm/skills-library/vectordb'
chroma_client = chromadb.PersistentClient(path=vectordb_path)
# Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_or_create_collection(
    name="test2", embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})
collection.peek()

{'ids': ['ADDWPT',
  'ADDWPTMODE',
  'AFTER',
  'ALT',
  'AREA',
  'AT',
  'ATALT',
  'ATDIST',
  'ATSPD',
  'BANK'],
 'embeddings': [[-0.008051572367548943,
   0.024463314563035965,
   -0.009314014576375484,
   0.011060391552746296,
   -0.039275962859392166,
   0.040145646780729294,
   0.0072730667889118195,
   0.026567384600639343,
   -0.04216555133461952,
   0.007518541533499956,
   0.03921985626220703,
   0.016131199896335602,
   -0.006501574534922838,
   0.0032630611676722765,
   -0.03983704745769501,
   -0.011432110331952572,
   -0.02962529845535755,
   0.0264130849391222,
   -0.02754928357899189,
   -0.0034524272195994854,
   0.006126348860561848,
   0.038041576743125916,
   -0.021517617627978325,
   0.015121245756745338,
   0.023439334705471992,
   -0.005379404406994581,
   0.009671705774962902,
   0.034450631588697433,
   0.005260173697024584,
   -0.00305090076290071,
   0.017239343374967575,
   0.0189225971698761,
   0.045756496489048004,
   0.011207676492631435,
   0.0168886

In [5]:
# capture output information from the bluesky client and return it as a string
@contextmanager
def capture_stdout():
    new_stdout = StringIO()
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield new_stdout
    finally:
        sys.stdout = old_stdout


def update_until_complete(client):
    complete_output = ""
    empty_output_count = 0  # Track consecutive empty outputs

    while True:
        with capture_stdout() as captured:
            client.update()
        new_output = captured.getvalue()

        # Check if the current output is empty
        if not new_output.strip():
            empty_output_count += 1  # Increment counter for empty outputs
        else:
            empty_output_count = 0  # Reset counter if output is not empty
            complete_output += new_output  # Add non-empty output to complete output

        # If there are two consecutive empty outputs, break the loop
        if empty_output_count >= 2:
            break

    # It's assumed you want to keep the last update outside the loop
    client.update()

    return complete_output

In [6]:
# connect to client
client = Client()
client.connect("127.0.0.1", 11000, 11001)
client.update()
client.update()

Client b'\x00\xd4!\xf2U' connected to host b'\x00\x7f\xf3\x88\x86'
Client active node changed.


In [7]:
with open('../prompts/basecmds.txt', 'r') as file:
    base_cmds = file.read()

Creating tools for agent

In [8]:

# get all aircraft info
# get conflict information

@tool
def GetAllAircraftInfo(command: str = 'GETACIDS'):
    """Get each aircraft information at current time: position, heading (deg), track (deg), altitude, V/S (vertical speed), calibrated, true and ground speed and mach number. Input is 'GETACIDS'.
    
    Parameters:
    - command: str (default 'GETACIDS')
    
    Example usage:
    - GetAllAircraftInfo('GETACIDS')
    
    Returns:
    - str: all aircraft information
    """
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    print(f'LLM input:{command}')
    
    client.send_event(b'STACK', command)
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output

@tool
def GetConflictInfo(commad: str = 'SHOWTCPA'):
    """Use this tool to identify and get vital information on aircraft pairs in conflict. It gives you Time to Closest Point of Approach (TCPA), Quadrantal Direction (QDR), separation distance, Closest Point of Approach distance (DCPA), and Time of Loss of Separation (tLOS).
    
    Parameters:
    - command: str (default 'SHOWTCPA')
    
    Example usage:
    - GetConflictInfo('SHOWTCPA')
    
    Returns:
    - str: conflict information between aircraft pairs
    """
    client.send_event(b'STACK', 'SHOWTCPA')
    time.sleep(1)
    sim_output = update_until_complete(client)
    return sim_output


@tool
def ContinueMonitoring(time: str = '5'):
    """Monitor for conflicts between aircraft pairs for a specified time. 
    Parameters:
    - time (str): The time in seconds to monitor for conflicts. Default is 5 seconds.
    
    Example usage:
    - ContinueMonitoring('5')
    
    Returns:
    - str: The conflict information between aircraft pairs throughout the monitoring period.
    """
    for i in range(int(time)):
        client.send_event(b'STACK', 'SHOWTCPA')
        time.sleep(1)
        sim_output += str(i) + ' sec: \n' + update_until_complete(client) + '\n'
    return sim_output


@tool
def SendCommand(command: str):
    """
    Sends a command with optional arguments to the simulator and returns the output. 
    You can only send 1 command at a time.
    
    Parameters:
    - command (str): The command to send to the simulator. Can only be a single command, with no AND or OR operators.
    
    Example usage:
    - SendCommand('COMMAND_NAME ARG1 ARG2 ARG3 ...) # this command requires arguments
    - SendCommand('COMMAND_NAME') # this command does not require arguments
    
    Returns:
    str: The output from the simulator.
    """
    # Convert the command and its arguments into a string to be sent
    #command_with_args = ' '.join([command] + [str(arg) for arg in args])
    # Send the command to the simulator
    # client.update()  # Uncomment this if you need to update the client state before sending the command
    print(command)
    # replace " or ' in the command string with nothing
    command = command.replace('"', '').replace("'", "")
    command = command.split('\n')[0]
    client.send_event(b'STACK', command)  
    # wait 1 second
    time.sleep(1)
    # Wait for and retrieve the output from the simulator
    sim_output = update_until_complete(client)
    if sim_output == '':
        return 'Command executed successfully.'
    if 'Unknown command' in sim_output:
        return sim_output + '\n' + 'Please use a tool QueryDatabase to search for the correct command.'
    return sim_output



@tool
def QueryDatabase(input: str):
    """If you want to send command to a simulator please first search which command you should use. For example if you want to create an aircraft, search for 'how do I create an aircraft'.
    Parameters:
    - input: str (the query to search for)
    Returns:
    - list: the top 5 results from the database
    """

    query_results = collection.query(
        query_texts=[input],
        n_results=5
    )

    return "The HDG command sets the aircraft's heading, disengaging LNAV mode. Use it by specifying the aircraft ID and desired heading in degrees, like: HDG acid, hdg_degrees. The aircraft ID is the unique identifier of the aircraft you want to control, and the heading is the desired direction in degrees. For example, to set the heading of aircraft ABC to 90 degrees, you would use the command: HDG ABC, 90 \n\n\n The ALT command adjusts an aircraft's altitude via autopilot, optionally setting vertical speed. Specify the aircraft ID, desired altitude in feet, and optionally, climb/descent speed in feet per minute, like ALT acid, alt, vspd. For example if you want to change aircraft KL123 height to 20000 ft the command is: ALT KL123 20000. or ALT KL123 FL200, or ALT KL123 FL200 10"
    #return query_results['documents'][0]


tools = [GetAllAircraftInfo, GetConflictInfo,
         SendCommand, QueryDatabase]

In [9]:
input = "check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading. Your Goal is to have no conflicts between aircraft."

In [10]:
# Example:
# Question: check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading. If there is no conflict then you don't need to do anything.
# Thought: I should check if there are any conflicts between aircraft with the GetConflictInfo tool.
# Action: GetConflictInfo
# Action Input: SHOWTCPA
# Observation: Conflict info ...
# Thought: There is a conflict between these aircraft .... Let me check their information
# Action: GetAllAircraftInfo
# Action Input: GETACIDS
# Observation: Aircraft info ...
# Thought: Based on ... I should ... because ... to resolve the conflict  
# Action: SendCommand
# Action Input: COMMAND_NAME ARG1 ARG2 ARG3 ...
# Observation: Command output ... I must confirm that the conflict is resolved
# Action: GetConflictInfo
# Action Input: SHOWTCPA
# Observation: Conflict info ...
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question


In [11]:
# Get the prompt to use - you can modify this!
from langchain import hub
react_prompt = hub.pull("hwchase17/react")
react_prompt.template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, specify exactly one of the following tools [{tool_names}]. No additional text or comments permitted.
Action Input: the input argument to the action.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question




Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
react_prompt.pretty_print()




Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, specify exactly one of the following tools [{tool_names}]. No additional text or comments permitted.
Action Input: the input argument to the action.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question




Begin!

Question: {input}
Thought:{agent_scratchpad}



In [12]:
xml_prompt = hub.pull("hwchase17/xml-agent-convo")
xml_prompt.pretty_print()

================================ Human Message =================================

You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer between <final_answer></final_answer>. For example:

<final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Previous Conversation:
{chat_history}

Question: {input}
{agent_scratchpad}


 Agent

In [13]:
# Get the prompt to use - you can modify this!
openai_function_prompt = hub.pull("hwchase17/openai-functions-agent")
openai_function_prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [15]:
import time
from langchain import hub







from langchain.agents import AgentExecutor, create_react_agent, create_json_chat_agent, create_xml_agent, create_openai_functions_agent







from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.chat_models import ChatOllama








from langchain_anthropic import AnthropicLLM, ChatAnthropic







from langchain_google_genai import GoogleGenerativeAI







# # anthropic llm







# llm = ChatAnthropic(







#     model='claude-3-opus-20240229')  # Choose the LLM to use







# llm = GoogleGenerativeAI(







#     model="gemini-pro", google_api_key=os.getenv("GEMINI_API_KEY"))
model_name = "mixtral"
llm = Ollama(base_url="https://ollama.junzis.com", model=model_name)
#llm = ChatOpenAI()
def invoke_until_success(llm):
    while True:
        try:
            # Attempt to invoke the method
            llm.invoke('hi')
            print("Invocation successful.")
            break  # Exit the loop if invocation was successful
        except Exception as e:
            print("Invocation failed, trying again...")
            time.sleep(1)  # Wait for a short period before retrying

# Example usage
# invoke_until_success(llm)
invoke_until_success(llm)








# Construct the ReAct agent








react_agent = create_react_agent(llm, tools, react_prompt)

xml_agent = create_xml_agent(llm, tools, xml_prompt)

openai_function_agent = create_openai_functions_agent(llm, tools, openai_function_prompt)







# Create an agent executor by passing in the agent and tools








react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True)

xml_agent_executor = AgentExecutor(
    agent=xml_agent, tools=tools, verbose=True, handle_parsing_errors=True)

openai_function_executor = AgentExecutor(
    agent=openai_function_agent, tools=tools, verbose=True, handle_parsing_errors=True)
#json_agent_executor = AgentExecutor(agent=json_agent, tools=tools, verbose=True, handle_parsing_errors=True)

Invocation failed, trying again...
Invocation failed, trying again...
Invocation failed, trying again...
Invocation failed, trying again...


In [212]:
client.send_event(b'STACK', 'IC simple/conflicts/2ac/case1.scn')

In [216]:
update_until_complete(client)

''

In [217]:
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project", tags=['openai', 'case 1 react']):
    out = react_agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...
I should first check for conflicts between aircraft using the GetConflictInfo tool. If conflicts are found, I can then use the SendCommand tool to change the altitude or heading of the aircraft to resolve the conflicts.
Action: GetConflictInfo
Action Input: 'SHOWTCPA'Aircraft Pairs in Conflict and their TCPA (sec):
KL204 - KL420 TCPA: 104.01 sec QDR: 270.00 deg Distance: 53256.30 m DCPA: 0.00 m tLOS: 85.93 sec
KL204 - KL420 TCPA: 104.01 sec QDR: 90.00 deg Distance: 53256.30 m DCPA: 0.00 m tLOS: 85.93 sec
I have identified conflicts between aircraft KL204 and KL420. Now, I need to change the altitude or heading of one or both aircraft to resolve the conflicts.
Action: SendCommand
Action Input: 'CHANGEALTITUDE KL204 10000''CHANGEALTITUDE KL204 10000'
Unknown command: CHANGEALTITUDE

Please use a tool QueryDatabase to search for the correct command.I should use the QueryDatabase tool to search for the correct command to change the altitude of an air

In [57]:

from langchain.agents import AgentExecutor, create_openai_tools_agent, OpenAIFunctionsAgent, create_react_agent

from langchain.memory import ConversationBufferMemory

# prompt = hub.pull("hwchase17/openai-tools-agent")
# memory = ConversationBufferMemory(
#     memory_key="chat_history", return_messages=True)

# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


chat_history = []

message = SystemMessage(
    content=(
        "You are very powerful air traffic controller, that can use commands to interact and control the airspace. "
        "You can use tools to get information about aircrafts - get_all_aircraft_info, conflicts - get_conflict_info, send commands to the simulator to interact with aircraft - send_command_to_simulator and get information and search for commands in the database - query_database. If you are going to use send_command_to_simulator tool you must first search for the command you should use in the database. Search: how can I ..."
    )
)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")],
)


# agent_kwargs = {
#     "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
# }

# # Create the memory
# memory = ConversationBufferMemory(memory_key="memory", return_messages=True)


# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, return_intermediate_steps=True)

In [70]:
user_input = "check if there are any conflicts between aircraft and if there are resolve them by changing altitude or heading"
while user_input != 'exit':
    if user_input != 'exit':
        # Invoke the agent and pass the current chat history
        update_until_complete(client)
        response = agent_executor.invoke(
            {"input": user_input, "chat_history": chat_history})
        human_message = HumanMessage(content=user_input)
        ai_message = AIMessage(content=response['output'])
        
        chat_history.append(human_message)
        chat_history.append(ai_message)
        print(AIMessage(content=response['output']))
        
        time.sleep(2)
        



> Entering new AgentExecutor chain...

Invoking: `get_conflict_info` with `{'command': 'SHOWTCPA'}`


No conflicts detected.
No conflicts have been detected between aircraft at the moment. All aircraft are currently maintaining safe separation. If you have any other requests or need further assistance, feel free to let me know!

> Finished chain.
content='No conflicts have been detected between aircraft at the moment. All aircraft are currently maintaining safe separation. If you have any other requests or need further assistance, feel free to let me know!'


> Entering new AgentExecutor chain...

Invoking: `get_conflict_info` with `{'command': 'SHOWTCPA'}`


Aircraft Pairs in Conflict and their TCPA (sec):
KL204 - KL420 TCPA: 91.01 sec QDR: 270.00 deg Distance: 46600.11 m DCPA: 0.00 m tLOS: 72.93 sec
KL204 - KL420 TCPA: 91.01 sec QDR: 90.00 deg Distance: 46600.11 m DCPA: 0.00 m tLOS: 72.93 sec

Invoking: `send_command_to_simulator` with `{'command': 'CHANGEALT', 'args': ['KL204', '2

KeyboardInterrupt: 

In [30]:
response['intermediate_steps'][-1][0].log

"\nInvoking: `send_command_to_simulator` with `{'command': 'COLOUR', 'args': ['KN00000', 'pink']}`\n\n\n"

In [147]:
agent_executor.invoke({'input': "what is my name?"})



> Entering new AgentExecutor chain...
I'm here to help you with any questions or tasks you have. If there's anything specific you'd like assistance with, feel free to let me know!

> Finished chain.


{'input': 'what is my name?',
 'memory': [HumanMessage(content='how many aircraft are in the airspace?'),
  AIMessage(content='There are 4 aircraft in the airspace with the following IDs: KN00000, KN00001, KN00002, KN00003.'),
  HumanMessage(content='what was the last answer?'),
  AIMessage(content='The last answer was a request for clarification. How can I assist you further?'),
  HumanMessage(content='what was your last answer?'),
  AIMessage(content='My last answer was a request to run multiple tools simultaneously in parallel. Is there anything specific you would like to know or ask about?'),
  HumanMessage(content="don't use tools, just answer how many aircraft were there"),
  AIMessage(content='There are 4 aircraft in the airspace.'),
  HumanMessage(content='my name is justas'),
  AIMessage(content='Hello Justas! How can I assist you today?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content="I'm here to help you with any questions or tasks you have. If there's any

In [6]:
# gpt-4-0125-preview
# gpt-3.5-turbo-0125
# gpt-3.5-turbo-instruct




llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)




#output_parser = StrOutputParser()
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm_with_tools = llm.bind_tools(tools)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [9]:
prompt_template = PromptTemplate.from_template(conflict_prompt)

In [10]:
input = prompt_template.format()
print(input)


Your are required to rovide the CPA (close point of approach - a point where 2 aircraft are closest to each other) and time to CPA between aircraf pairs. The output should be like this: aircraft peer, CPA (latitude,longitude), CPA time, explanation
 If there are more than 2 aircraft involved, provide information for each pair.

example of a final answer:

aircraft: KL200 - KL400

CPA: 50,40

CPA time: 2024-03-20 01:28:39

Explanation:because this and that ...




define a new waypoint with the name of aircraft pairs in the simulation indicating where the CPA point(s) are located (you can query a database to check how to do that)


Use a Python programming tool for that for calculating precise locations and times. You cannot assume any numbers, it must be programatically calculated. It does not matter if it takes a long time, just do it. 








In [12]:
client.update()


In [13]:
out = agent_executor.invoke({"input": input})




> Entering new AgentExecutor chain...

Invoking: `get_all_aircraft_info` with `{'command': 'GETACIDS'}`


B2
Aircraft ids: ['A1', 'B2']
Info on A1 A320 index = 0
Pos: N53'14'0.0"  E002'24'16.0"
Hdg: 168   Trk: 168
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300

Info on B2 A320 index = 1
Pos: N51'56'10.0"  E002'50'36.0"
Hdg: 343   Trk: 343
Alt: 15000 ft  V/S: 0 fpm
CAS/TAS/GS: 150/188/188 kts   M: 0.300


Invoking: `e2b_data_analysis` with `{'python_code': 'from datetime import datetime, timedelta\nimport numpy as np\nfrom scipy.optimize import minimize\n\ndef lat_lon_to_cartesian(lat, lon, alt):\n    # Convert latitude and longitude to radians\n    lat = np.radians(lat)\n    lon = np.radians(lon)\n    # Earth\'s radius in meters (mean radius)\n    R = 6371000 + alt\n    # Convert to Cartesian coordinates\n    x = R * np.cos(lat) * np.cos(lon)\n    y = R * np.cos(lat) * np.sin(lon)\n    z = R * np.sin(lat)\n    return np.array([x, y, z])\n\ndef calculate_cpa(aircraft

In [21]:
out["output"]

'Based on the information retrieved, we have two aircraft, KL204 and KL420, both at the same altitude of 25,000 ft. KL204 is heading west (270 degrees) and KL420 is heading east (90 degrees). Both aircraft are at the same latitude (N52\'00\'0.0") but at significantly different longitudes (KL204 at W008\'00\'22.0" and KL420 at E015\'12\'22.0"). Given their headings and positions, they are moving towards each other, which could potentially lead to a conflict.\n\nTo accurately determine if there will be a Closest Point of Approach (CPA) that poses a conflict and, if so, calculate the CPA and the time to CPA, we need to analyze their speeds and headings in more detail. However, based on the provided information, there is a potential conflict due to their opposing headings at the same altitude and latitude.\n\nFINAL ANSWER:\n\naircraft: KL204 - KL420\n\nCPA: To be calculated\n\nCPA time: To be calculated\n\nExplanation: Both aircraft are at the same altitude and latitude, heading directly t